<a href="https://colab.research.google.com/github/RyuichiSaito1/covid19-twitter-usa-restoring/blob/main/full_archive_search_nyc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pandas

In [ ]:
# Stay-at-Home-Ordered

import datetime

# start_date = datetime.date(2021, 10, 31)
start_date = datetime.date(2021, 11, 21)
# end_date = datetime.date(2022, 12, 31)
end_date = datetime.date(2022, 1, 1)

city = 'new york'

search_strings = ['"go out"', '"going out"',
                  '"go to work"', '"going to work"',
                  '"to my office"', '"to school"',
                  '"to college"', '"attend the class"',
                  '"by train"', '"by bus"']

directory1 = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/1/1/'
directory2 = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/1/2/'


In [ ]:
# Restrictions-on-Gatherings

import datetime

start_date = datetime.date(2021, 10, 31)
end_date = datetime.date(2022, 12, 31)

city = 'new york'

search_strings = ['"to the movies"', '"to the theater"',
                  '"see a play"', '"to a concert"',
                  '"to the concert"', '"to the gig"',
                  '"to the museum"', '"to the planetarium"',
                  '"see the game"', '"to the stadium"',
                  '"gathering"', '"to a club"',
                  '"to the amusement"', '"go bowling"',
                  '"to casinos"', '"to the racetrack"',
                  '"to the bingo"', '"to an arcade"',
                  '"to the arcade"', '"to church"',
                  '"wedding"', '"on a date"',
                  '"to a party"', '"to the party"',
                  '"have a party"', '"to bbq"',
                  '"hava a bbq"', '"for a drink"',
                  '"to a bar"', '"get-together"',
                  '"drinking party"', '"to a restaurant"',
                  '"eat out"', '"eating out"',
                  '"to a cafe"', '"meetup"',
                  '"go shopping"', '"going shopping"',
                  '"to the mall"', '"to the salon"',
                  '"get a haircut"', '"to the gym"',
                  '"go swimming"', '"to a spa"',
                  '"to a spa"', '"to yoga"',
                  '"to massage"', '"to the auditorium"']

directory1 = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/2/1/'
directory2 = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/2/2/'

In [ ]:
# Travel-Restrictions

import datetime

start_date = datetime.date(2021, 10, 31)
end_date = datetime.date(2022, 12, 31)

city = 'new york'

search_strings = ['"travel"', '"traveling"',
                  '"on a trip"', '"fly to"',
                  '"by plane"', '"visit"',
                  '"to my hometown"']

directory1 = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/3/1/'
directory2 = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/3/2/'


In [ ]:
# https://towardsdatascience.com/basic-tweet-preprocessing-in-python-efd8360d529e

from datetime import timedelta
import time
import requests
import os
import json
import pandas
import re

def replace_number_to_zero(text):
    changed_text = re.sub(r'[0-9]+', "0", text)
    return changed_text

def date_range(start, stop, step = timedelta(1)):
    current = start
    while current <= stop:
        yield current
        current += step

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

bearer_token = "AAAAAAAAAAAAAAAAAAAAABeVOwEAAAAAjo5fJzFQCqV1bkwOCT89XAIPtqE%3DqcBXeNvTsRnR7zvitW2R0nazcHF230wtCKSghdujOIovWZ9C93"

search_url = "https://api.twitter.com/2/tweets/search/all"

def main():

    columns = ['author_id', 'created_at','id', 'text', 'geo']
    df = pandas.DataFrame(columns=columns)

    columns2 = ['id', 'name','username']
    df2 = pandas.DataFrame(columns=columns2)

    headers = create_headers(bearer_token)

    # Repeat for dates
    i = 0
    for date in date_range(start_date, end_date):

      # Search Block
      search_date = date.strftime('%Y-%m-%d')
      from_date = search_date + 'T00:00:01Z'
      to_date =search_date + 'T23:59:59Z'

      query_params = ""

      for j in range(0, len(search_strings), 2):

        if j + 1 < len(search_strings):
          query = f"({search_strings[j]} OR {search_strings[j+1]}) -is:retweet point_radius:[-74.005880 40.712791 25mi] lang:en"
        else:
          query = f"{search_strings[j]} -is:retweet point_radius:[-74.005880 40.712791 25mi] lang:en"

        query_params = {
          'query': query,
          'start_time': from_date,
          'end_time': to_date,
          'place.fields': 'geo',
          'tweet.fields': 'author_id,created_at,geo',
          'expansions': 'author_id',
          'max_results': '500'}  # between 10 and 500
        print(query_params)

        json_response = connect_to_endpoint(search_url, headers, query_params)
        print(json.dumps(json_response, indent=4, sort_keys=True))

        print('Wait 8 seconds')
        time.sleep(8)

        if json_response['meta']['result_count'] != 0:
          # continue

          # Output Block
          # Repeat for number of acquisitions
          for tweet in json_response['data']:

            author_id = tweet['author_id']
            created_at = tweet['created_at']
            id = tweet['id']
            text = tweet['text']
            try:
              geo = tweet['geo']
            except KeyError:
              geo =''
            concat_list = [author_id, created_at, id, text, geo]
            df_next = pandas.DataFrame([concat_list], columns=columns)
            # df = df.concat(df_next)
            df = pandas.concat([df, df_next])

          # Repeat for number of acquisitions
          for tweet in json_response['includes']['users']:

            id = tweet['id']
            user = tweet['name']
            username = tweet['username']
            concat_list = [id, user, username]
            df_next = pandas.DataFrame([concat_list], columns=columns2)
            # df2 = df2.concat(df_next)
            df2 = pandas.concat([df2, df_next])

      i += 1

      if i == 7:
        # Preprocessing
        df = df.replace('(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+$,%#]+)', '', regex=True)
        df = df.replace('(@[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+$,%#]+)', '', regex=True)
        df = df.replace('(#[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+$,%#]+)', '', regex=True)
        df.text = df.text.replace('\n', '', regex=True)
        df.text = df.text.replace('\r', '', regex=True)
        df.text = df.text.replace('amp;', '', regex=True)
        df.text = df.text.replace('RT', '', regex=True)
        df.text = df.text.replace('FAV', '', regex=True)
        df.text = df.text.map(replace_number_to_zero)
        df = df.query('text != " "')
        from_date = date.strftime('%Y-%m-%d')
        from_date = date + timedelta(days = -6)
        from_date = from_date.strftime('%Y-%m-%d')
        to_date = date.strftime('%Y-%m-%d')

        directory = directory1 + from_date + '_' + to_date + '_' + city + '.tsv'
        print('Save: ' + directory)
        df.to_csv(directory, sep='\t', index=False, header=False)

        directory = directory2 + from_date + '_' + to_date + '_' + city + '.tsv'
        print('Save: ' + directory)
        df2.to_csv(directory, sep='\t', index=False, header=False)

        print('Wait 4 seconds')
        time.sleep(4)
        # Initialize the Data Frame on day 7
        df = pandas.DataFrame(columns=columns)
        df2 = pandas.DataFrame(columns=columns2)
        # Initialize the counter on day 7
        i = 0

if __name__ == "__main__":
    main()